In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.legend_handler

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 7
plt.rcParams['lines.linewidth'] = .5

%matplotlib inline

In [ ]:
library_df =pd.read_excel('../data_tables/crispey3_libraries_by_pool_number.xlsx')

In [ ]:
counts = pd.read_csv('../data_tables/mapped_counts_pbc_umi_TBF_SC.tsv', '\t')
counts['well'] = counts.barcode_id.str.split('_').str[0].astype(float)
counts = counts[counts['well'].isin(library_df['ergosterol'].dropna().tolist())]
for col in counts.filter(regex = 'counts').columns:
    counts[col] = (counts[col]/counts[col].sum())*1000000

In [ ]:
def raw_count_fraction_plotter(barcode_list, condition, color, generations):
    if color == 'red':
        colors = ['red','firebrick']
    elif color == 'royalblue':
        colors = ['deepskyblue','blue', 'navy']
    else: 
        colors = [color]*len(barcode_list)
    if color == 'black':
        colors = ['black','darkgrey','slategrey','lightslategrey','lightgrey','dimgray',]
        alpha = .3
    else:
        alpha = .9
    hit = counts[counts['barcode_id'].isin(barcode_list)].filter(regex = condition+'|barcode_id')
    gens = [5,10,15,20,25,30]
    for i,barcode in enumerate(barcode_list):
        bc = hit[hit['barcode_id']==barcode].filter(regex = condition).T
        cc = colors[i]
        for col in bc.columns:
            for flask in ['F1','F2','F3']:
                gens = generations[flask]
                gens_for_plot  = [i-gens[0] for i in gens]
                plt.plot(gens_for_plot, (bc.loc[bc.index.str.contains(flask), col])/(bc.loc[bc.index.str.contains(flask), col].values[0]), c= cc, alpha = alpha)
     
        

In [ ]:

fig, ax = plt.subplots(figsize = (2.1,2.0))
generations = {'F1':[4.7830620088,10.0796047852,15.3915487915,20.6512920552,26.309503538,31.5670410456],
              'F2':[4.8673622314,10.3251737544,15.5862196071,20.845606236,26.5411052128,31.7249571723],
              'F3':[4.8567048384,10.3777886461,15.7608693786,20.9214261975,26.3615721454,31.6419211365]}
raw_count_fraction_plotter(['012_009','012_010'], 'TBF','red', generations)
raw_count_fraction_plotter(['002_012','002_013'], 'TBF', 'royalblue', generations)
raw_count_fraction_plotter(['033_117','015_119','032_117','018_119','027_111','027_113'], 'TBF', 'black', generations)
plt.xticks([0,5,10,15,20,25])
plt.xlim(0,26)

custom_lines = [(Line2D([0], [0], color='red', lw=4),
                Line2D([0], [0], color='firebrick', lw=4)),
                (Line2D([0], [0], color='black', lw=4),
                Line2D([0], [0], color='darkgrey', lw=4),
                Line2D([0], [0], color='lightgrey', lw=4),
                Line2D([0], [0], color='slategrey', lw=4),
                Line2D([0], [0], color='dimgrey', lw=4),
                Line2D([0], [0], color='lightslategrey', lw=4)), 
                (Line2D([0], [0], color='deepskyblue', lw=4),
               Line2D([0], [0], color='blue', lw=4))]
leg = ax.legend(handles = custom_lines, handlelength = 3, title = '  BCs   Chr7 Variants', labels = ['847050C>A', 'No-edit', '848783AC>A'],
           handler_map = {tuple: matplotlib.legend_handler.HandlerTuple(None)}, loc = 'lower left', bbox_to_anchor = (-.01,-.01), fontsize = 5,title_fontsize=5)
leg._legend_box.align = "left"
plt.yscale('log')
plt.ylim([.08,10])
plt.xticks(fontsize = 7)
plt.yticks(fontsize = 7)
plt.xlabel('Time (generations)', fontsize = 7)
plt.ylabel('Log Fold Change in Abundance', fontsize = 7)
plt.savefig('../GxE_Figures/Figure_1/figure1g.svg')
